## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-19-20-54-05 +0000,nypost,Disgraced ex-California deputy sheriff who kil...,https://nypost.com/2026/01/19/us-news/ex-calif...
1,2026-01-19-20-52-36 +0000,nyt,Federal Reserve Inquiry Clouds Trump’s Supreme...,https://www.nytimes.com/2026/01/19/us/politics...
2,2026-01-19-20-52-32 +0000,nyt,How 40 People Died In a Train Crash in Andaluc...,https://www.nytimes.com/2026/01/19/world/europ...
3,2026-01-19-20-51-27 +0000,bbc,At the scene of Spain's worst rail disaster in...,https://www.bbc.com/news/videos/cn82xejl4e2o?a...
4,2026-01-19-20-49-53 +0000,nypost,Blockhead anti-ICE protesters swarm Minneapoli...,https://nypost.com/2026/01/19/us-news/anti-ice...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2354/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,trump,62
79,greenland,26
27,spain,14
245,least,14
190,new,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
152,2026-01-19-12-12-55 +0000,nypost,Trump says he no longer needs to ‘think purely...,https://nypost.com/2026/01/19/us-news/trump-sa...,130
151,2026-01-19-12-13-28 +0000,cbc,European leaders push back on Trump's tariff t...,https://www.cbc.ca/news/world/greenland-europe...,121
238,2026-01-19-02-38-00 +0000,wsj,Precious Metals Climb as Trump Plans Tariff on...,https://www.wsj.com/finance/precious-metals-cl...,121
27,2026-01-19-19-44-02 +0000,nyt,European Allies Scramble to Push Back Against ...,https://www.nytimes.com/2026/01/19/us/politics...,117
50,2026-01-19-18-33-55 +0000,nypost,European stocks fall sharply after Trump threa...,https://nypost.com/2026/01/19/business/europea...,116


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
152,130,2026-01-19-12-12-55 +0000,nypost,Trump says he no longer needs to ‘think purely...,https://nypost.com/2026/01/19/us-news/trump-sa...
259,82,2026-01-19-00-00-06 +0000,nypost,At least 20 killed in high-speed train derailm...,https://nypost.com/2026/01/18/world-news/at-le...
52,46,2026-01-19-18-32-03 +0000,startribune,The latest: Department of Justice plans to app...,https://www.startribune.com/ice-raids-minnesot...
139,42,2026-01-19-13-17-00 +0000,wsj,European Stock Markets Tumble on Trump’s Tarif...,https://www.wsj.com/finance/stocks/global-equi...
142,40,2026-01-19-13-00-00 +0000,wsj,Half of U.S. voters say the Trump administrati...,https://www.wsj.com/politics/policy/voters-ske...
240,36,2026-01-19-02-25-22 +0000,nypost,"Iran threatens to resume executions, warns of ...",https://nypost.com/2026/01/18/world-news/iran-...
117,35,2026-01-19-14-52-09 +0000,nypost,Police probe if dead backpacker was ‘attacked’...,https://nypost.com/2026/01/19/world-news/polic...
162,32,2026-01-19-11-41-57 +0000,bbc,Ecstasy and now a day off - Senegal fans aroun...,https://www.bbc.com/news/articles/ce3ekz4yx21o...
173,31,2026-01-19-11-00-46 +0000,wapo,Civil rights leaders see another turning point...,https://www.washingtonpost.com/nation/interact...
217,30,2026-01-19-06-40-23 +0000,bbc,China hits growth goal after exports defy US t...,https://www.bbc.com/news/articles/cgk8zd287myo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
